# Churn Modelling using Keras

In [1]:
import pandas as pd
#setting pandas to show all columns
pd.set_option('display.max_columns', None)

In [2]:
#Loading the CSV into a Data Frame Object
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

### Let's take a look at our Data

In [3]:
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


CustomerID does not tell us anything, therefore we can delete it from the Data Frame

In [4]:
df.drop('customerID', axis=1, inplace=True)

With the exception of <b>Tenure</b>, <b>TotalCharges</b>, <b>MonthlyCharges</b> and <b>Churn</b> all the other variables are <b>Categorical</b>, so we should get their dummie variables.<br>
Always remember to drop the first dummie colums for two basic reasons:<br>
- It is always possible to <b>infer</b> the values you dropped given the values you kept.<br>
- To avoid the <b>Dummie Variable Trap</b>.<br>
If you don't know what that is, take a look at this webpage:<br>
<a>http://www.algosome.com/articles/dummy-variable-trap-regression.html<a>

In [5]:
X = pd.get_dummies(df.drop(['tenure', 'MonthlyCharges', 'TotalCharges', 'Churn'], axis=1), drop_first=True)
#Putting together our independent variables
X = pd.concat([X, df[['tenure', 'MonthlyCharges', 'TotalCharges']]], axis=1)

In [6]:
#Our dependent variables are basically the df 'Churn' column
y = pd.get_dummies(df['Churn'], drop_first=True).values.ravel()

### Taking a deeper look

In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 30 columns):
SeniorCitizen                            7043 non-null int64
gender_Male                              7043 non-null uint8
Partner_Yes                              7043 non-null uint8
Dependents_Yes                           7043 non-null uint8
PhoneService_Yes                         7043 non-null uint8
MultipleLines_No phone service           7043 non-null uint8
MultipleLines_Yes                        7043 non-null uint8
InternetService_Fiber optic              7043 non-null uint8
InternetService_No                       7043 non-null uint8
OnlineSecurity_No internet service       7043 non-null uint8
OnlineSecurity_Yes                       7043 non-null uint8
OnlineBackup_No internet service         7043 non-null uint8
OnlineBackup_Yes                         7043 non-null uint8
DeviceProtection_No internet service     7043 non-null uint8
DeviceProtection_Yes                   

Looking carefully it is possible to see that <b>TotalCharges</b> is being treated as a <b>Object</b> and we need it to be a <b>Float</b>.<br>
So, let's us take care of that.

In [8]:
X['TotalCharges'] = pd.to_numeric(X['TotalCharges'], errors='coerce')
X['TotalCharges'].fillna(0, inplace=True)

In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 30 columns):
SeniorCitizen                            7043 non-null int64
gender_Male                              7043 non-null uint8
Partner_Yes                              7043 non-null uint8
Dependents_Yes                           7043 non-null uint8
PhoneService_Yes                         7043 non-null uint8
MultipleLines_No phone service           7043 non-null uint8
MultipleLines_Yes                        7043 non-null uint8
InternetService_Fiber optic              7043 non-null uint8
InternetService_No                       7043 non-null uint8
OnlineSecurity_No internet service       7043 non-null uint8
OnlineSecurity_Yes                       7043 non-null uint8
OnlineBackup_No internet service         7043 non-null uint8
OnlineBackup_Yes                         7043 non-null uint8
DeviceProtection_No internet service     7043 non-null uint8
DeviceProtection_Yes                   

<b>Done!</b><br>Now we can move on to the next step.

### Scaling the independent variables

It is always a common practice to scale our data, specially if there are some values in it that would make the smaller values seem insignificant.

In [10]:
from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
X = scl.fit_transform(X)

It is, however, unnecessary to scale the dependent variables. Cause that wouldn't add any value to our model since we're expecting a binary outcome.

### Spliting into Train and Test sets

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=101)

## Now what?

Now that we have our <b>train</b> and <b>test</b> data, we must decide which way to go.<br>
We could simply <b>fit</b> the model to the train data and make some <b>predictions</b> with it, it could even achieve a <b>high accuracy</b>. But who can garantee that its accuracy wasn't <b>just a coincidence</b>?<br><br>
We'll get to that later on.

### The simplest way

In [12]:
#import keras
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


Why use <b>Keras</b>?<br>
<b>Keras</b> is a <b>High Level API</b> written in <b>Python</b> and it is, above all, <b>very easy</b> to learn and implement. <br><br>
So... let us proceed.

In [13]:
#creating the Model
model = Sequential()
model.add(Dense(units=16, kernel_initializer='uniform', activation='relu', input_dim=30)) # first hidden layer
model.add(Dense(units=16, kernel_initializer='uniform', activation='relu')) # second hidden layer
model.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid')) # output layer
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) #compiling the model

### What the heck happend here?!
<br>
<b>Sequential</b> model is a linear stack of layers.<br>
<b>Dense</b> is a Densely-connected Neural Network Layer. It makes it incredibly easy to add layers to our NN.<br>
The <b>units</b> parameter is the number of neurons in the current layer. It is a rule of thumb to use as it's value the sum of inputs and outputs divided by two. But, feel free to mess around with it and see what happens.<br>
The <b>activation</b> parameter is the activation function to use in that particular layer.<br>
The <b>input_dim</b> parameter is the dimension of the input layer.<br>
<b>Compile</b> is the method that states the learning process of our model. It does so by defining an optimizer, a loss function (what the NN tries to minimize) and a list of metrics.<br><br>
For more info, take a look at Keras documentation:<br>
<a>https://keras.io/getting-started/sequential-model-guide/<a>

Now that our model is <b>built</b>, we should <b>fit</b> it.

In [14]:
model.fit(X_train, y_train, batch_size=20, epochs=300, verbose=1)

Epoch 1/300
5634/5634 [==============================] - 0s 76us/step - loss: 0.5173 - acc: 0.7638
Epoch 2/300
5634/5634 [==============================] - 0s 47us/step - loss: 0.4214 - acc: 0.8024
Epoch 3/300
5634/5634 [==============================] - 0s 46us/step - loss: 0.4157 - acc: 0.8042
Epoch 4/300
5634/5634 [==============================] - 0s 32us/step - loss: 0.4130 - acc: 0.8016
Epoch 5/300
5634/5634 [==============================] - 0s 32us/step - loss: 0.4106 - acc: 0.8053
Epoch 6/300
5634/5634 [==============================] - 0s 31us/step - loss: 0.4102 - acc: 0.8035
Epoch 7/300
5634/5634 [==============================] - 0s 36us/step - loss: 0.4095 - acc: 0.8065
Epoch 8/300
5634/5634 [==============================] - 0s 34us/step - loss: 0.4088 - acc: 0.8064
Epoch 9/300
5634/5634 [==============================] - 0s 31us/step - loss: 0.4079 - acc: 0.8106
Epoch 10/300
5634/5634 [==============================] - 0s 31us/step - loss: 0.4075 - acc: 0.8076
Epoch 11/

5634/5634 [==============================] - 0s 32us/step - loss: 0.3721 - acc: 0.8312
Epoch 83/300
5634/5634 [==============================] - 0s 32us/step - loss: 0.3721 - acc: 0.8307
Epoch 84/300
5634/5634 [==============================] - 0s 31us/step - loss: 0.3719 - acc: 0.8287
Epoch 85/300
5634/5634 [==============================] - 0s 35us/step - loss: 0.3720 - acc: 0.8284
Epoch 86/300
5634/5634 [==============================] - 0s 42us/step - loss: 0.3715 - acc: 0.8321
Epoch 87/300
5634/5634 [==============================] - 0s 41us/step - loss: 0.3707 - acc: 0.8321
Epoch 88/300
5634/5634 [==============================] - 0s 31us/step - loss: 0.3704 - acc: 0.8303
Epoch 89/300
5634/5634 [==============================] - 0s 31us/step - loss: 0.3710 - acc: 0.8308
Epoch 90/300
5634/5634 [==============================] - 0s 33us/step - loss: 0.3703 - acc: 0.8303
Epoch 91/300
5634/5634 [==============================] - 0s 32us/step - loss: 0.3700 - acc: 0.8307
Epoch 92/300


5634/5634 [==============================] - 0s 31us/step - loss: 0.3596 - acc: 0.8371
Epoch 163/300
5634/5634 [==============================] - 0s 34us/step - loss: 0.3606 - acc: 0.8367
Epoch 164/300
5634/5634 [==============================] - 0s 31us/step - loss: 0.3605 - acc: 0.8340
Epoch 165/300
5634/5634 [==============================] - 0s 32us/step - loss: 0.3600 - acc: 0.8365
Epoch 166/300
5634/5634 [==============================] - 0s 31us/step - loss: 0.3593 - acc: 0.8395
Epoch 167/300
5634/5634 [==============================] - 0s 31us/step - loss: 0.3593 - acc: 0.8376
Epoch 168/300
5634/5634 [==============================] - 0s 34us/step - loss: 0.3591 - acc: 0.8401
Epoch 169/300
5634/5634 [==============================] - 0s 47us/step - loss: 0.3595 - acc: 0.8367
Epoch 170/300
5634/5634 [==============================] - 0s 43us/step - loss: 0.3597 - acc: 0.8387
Epoch 171/300
5634/5634 [==============================] - 0s 41us/step - loss: 0.3592 - acc: 0.8364
Epoc

Epoch 243/300
5634/5634 [==============================] - 0s 32us/step - loss: 0.3509 - acc: 0.8404
Epoch 244/300
5634/5634 [==============================] - 0s 32us/step - loss: 0.3514 - acc: 0.8413
Epoch 245/300
5634/5634 [==============================] - 0s 33us/step - loss: 0.3506 - acc: 0.8403
Epoch 246/300
5634/5634 [==============================] - 0s 32us/step - loss: 0.3493 - acc: 0.8419
Epoch 247/300
5634/5634 [==============================] - 0s 32us/step - loss: 0.3506 - acc: 0.8408
Epoch 248/300
5634/5634 [==============================] - 0s 32us/step - loss: 0.3508 - acc: 0.8397
Epoch 249/300
5634/5634 [==============================] - 0s 34us/step - loss: 0.3508 - acc: 0.8408
Epoch 250/300
5634/5634 [==============================] - 0s 49us/step - loss: 0.3499 - acc: 0.8442
Epoch 251/300
5634/5634 [==============================] - 0s 44us/step - loss: 0.3501 - acc: 0.8390
Epoch 252/300
5634/5634 [==============================] - 0s 32us/step - loss: 0.3501 - ac

And now let's make some <b>predictions</b>.

In [15]:
y_pred = model.predict(X_test)

### Is that it?
<br>
Hold your horses, cowboy!<br>
If you are anywhat familiar with the math of a <b>Sigmoid Function</b> you should know that a sigmoid outputs the likelihood of some event.<br>
If we take a look at our <b>y_pred</b> variable, we would see that it has the probabilities of a customer churn.<br>
To see how our model performed, however, we should have a <b>True</b>-<b>False</b> variable. And to do so, we must define a threshold to decide when a value becomes 0 and when it becomes 1.<br><br>
Our threshold here will be <b>.5</b>

In [16]:
y_pred = (y_pred > 0.5)

### To the performance!

In [17]:
from sklearn.metrics import confusion_matrix, classification_report
print (confusion_matrix(y_test, y_pred))
print ('\n')
print (classification_report(y_test, y_pred))

[[912 114]
 [192 191]]


             precision    recall  f1-score   support

          0       0.83      0.89      0.86      1026
          1       0.63      0.50      0.56       383

avg / total       0.77      0.78      0.77      1409



### Wait a second...
<br>
We got a relatively <b>lower accuracy</b> in the test set than in the train set. But <b>why</b>?<br>
It could mean two things:<br>
<b>Overfitting</b>, or...<br>
Remember when I said earlier that it <b>wasn't a good idea</b> to just simply train the model? Well, I didn't. But I never said that it was a good one either.<br>
The <b>accuracy</b> obtained while training the model could have been just a <b>coincidence</b>, just as I said.<br><br>
One way to prevent that is to use SciKit-learn's <b>Cross Validation</b>.<br><br>
So let's do just that.

In [18]:
from sklearn.model_selection import cross_val_score

In order to use the <b>Cross Validation</b>, we should observe a couple things:<br><br>
<li>
The Cross Validation expects, as a parameter, an estimator.
</li>
<li>
But we can't just feed him the model we've created.<br><br>
</li>    
So, we'll use a <b>wrapper</b> from keras. It will give us a model that we can use with the Cross Validation.

In [19]:
from keras.wrappers.scikit_learn import KerasClassifier

The KerasClassifier also need shomething:<br>
A <b>building function</b>.<br>
We basically repeat our steps to build the NN. But within a function.

In [20]:
def build_model():
    #creating the Model
    model = Sequential()
    model.add(Dense(units=16, kernel_initializer='uniform', activation='relu', input_dim=30)) # first hidden layer
    model.add(Dense(units=16, kernel_initializer='uniform', activation='relu')) # second hidden layer
    model.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid')) # output layer
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) #compiling the model
    return model

Now we instantiate a new <b>Keras Classifier</b> using our <b>building function</b>.<br>
And then, we are good to go.

In [23]:
model = KerasClassifier(build_fn=build_model, batch_size=20, epochs=300, verbose=0)
accuracies = cross_val_score(model, X_train, y_train, cv=10, verbose=0)

In [24]:
print ('Mean: ', accuracies.mean(), '-- Std: ', accuracies.std())

Mean:  0.7825636480957596 -- Std:  0.012888437350206896
